In [1]:
import pandas as pd

In [3]:
final_arousal_df = pd.read_csv('Final_Arousal_Features.csv')
final_valence_df = pd.read_csv('Final_Valence_Features.csv')

In [20]:
final_valence_df.drop(["Arousal","Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)
final_arousal_df.drop(["Valence", "Gender", "participant", "video", "Video_Name", "Age"], inplace=True, axis=1)